In [ ]:
!pip install -U -q kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#import os
#os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"
#%cd /content/gdrive/My Drive/Kaggle
#!kaggle datasets download -d jessicali9530/celeba-dataset

In [ ]:
%cd /content/

/content


In [ ]:
!unzip -q "./gdrive/My Drive/Kaggle/celeba-dataset.zip"

In [ ]:
# imports
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.layers as layers
import os
import numpy as np

In [ ]:
# some cool constants
IMG_SIZE = (128, 128, 3)
TRAIN_BATCH_SIZE = 32
VAL_BATCH_SIZE = 4
LATENT_VECTOR_SIZE = 200

In [ ]:
# load dataset
def build_dataset(img_fp, train_batch_size, val_batch_size, shuffle=True, validation_split=487/6331):
  """ build the dataset (all training as we only need loss) from img_filepath """
  data_generator = keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=validation_split)
  train_dataset = data_generator.flow_from_directory(img_fp, 
                                                    target_size = IMG_SIZE[:2],
                                                    batch_size = train_batch_size,
                                                    shuffle = shuffle,
                                                    class_mode=None,
                                                    subset = 'training'
                                                    )
  val_dataset = data_generator.flow_from_directory(img_fp, 
                                                    target_size = IMG_SIZE[:2],
                                                    batch_size = val_batch_size,
                                                    shuffle = False,
                                                    class_mode=None,
                                                    subset = 'validation'
                                                    )
  return train_dataset, val_dataset

In [ ]:
# load dataset
IMGS_FP = "./img_align_celeba/"
IMGS_FP_FULL = IMGS_FP + "img_align_celeba/"
LIST_OF_FILES = os.listdir(IMGS_FP_FULL)
NUM_IMGS = len(LIST_OF_FILES)
print("There are %s celebs in the dataset" % NUM_IMGS)
while NUM_IMGS % TRAIN_BATCH_SIZE != 0: # lazy solution to batch size not dividing num_datapoints
  print("file removed")
  os.remove(IMGS_FP + "img_align_celeba/" + os.listdir(IMGS_FP + "img_align_celeba")[-1])
  LIST_OF_FILES = os.listdir(IMGS_FP_FULL)
  NUM_IMGS = len(LIST_OF_FILES)

print(len(os.listdir(IMGS_FP_FULL)))
train_dataset, val_dataset = build_dataset(IMGS_FP, TRAIN_BATCH_SIZE, VAL_BATCH_SIZE)

There are 202599 celebs in the dataset
file removed
file removed
file removed
file removed
file removed
file removed
file removed
202592
Found 187008 images belonging to 1 classes.
Found 15584 images belonging to 1 classes.


In [ ]:
print(tf.shape(train_dataset[0]))

tf.Tensor([ 32 128 128   3], shape=(4,), dtype=int32)


In [ ]:
# Define encoder
def build_encoder(input_size, output_size):
  """ build the encoder network"""
  encoder_input = layers.Input(input_size, name='encoder_input')

  # 3 conv + relu
  current_layer = encoder_input
  for i in range(3):
      current_layer = layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same', activation='relu', name='conv_' + str(i))(current_layer)

  # 1 conv and sigmoid
  current_layer = layers.Conv2D(64, (3,3), strides=(2,2), padding="same", activation='sigmoid', name='conv_4')(current_layer) # squish latent into [0, 1]

  # flatten
  size_before_flat = current_layer.shape[1:]  # remember this shape, so decoder can build back up
  flat_layer = layers.Flatten(name='flat')(current_layer)
  encoder_output = layers.Dense(output_size, name='latent')(flat_layer)

  # make a keras Model
  encoder_network = keras.models.Model(encoder_input, encoder_output)

  return encoder_network, size_before_flat

In [ ]:
# define decoder
def build_decoder(input_size, size_before_flat, output_size):
  """ build the encoder network"""
  decoder_input = layers.Input(input_size, name='input')

  # Dense back up to size_before_flat
  current_layer = layers.Dense(np.prod(size_before_flat), name='toReshape')(decoder_input)
  current_layer = layers.Reshape(size_before_flat, name='reshape')(current_layer)

  # 3 conv_transpose + relu
  for i in range(3):
    current_layer = layers.Conv2DTranspose(64, (3,3), strides=(2, 2), padding='same', activation='relu', name='conv_' + str(i))(current_layer)

  # last layer has 3 conv filters to make depth=3 (RGB), sigmoid to squish into [0, 1]
  current_layer = layers.Conv2DTranspose(3, (3,3), strides=(2, 2), padding='same', activation='sigmoid', name='conv_4')(current_layer)

  decoder_output = current_layer
  # make a keras Model
  decoder_network = keras.models.Model(decoder_input, decoder_output)

  return decoder_network

In [ ]:
# class for the AE Model. Done this way so loss easier to calculate
class AE(keras.Model):
  def __init__(self, input_size, latent_size, **kwargs):
      super(AE, self).__init__(**kwargs)
      self.encoder, size_before_flat = build_encoder(input_size, latent_size)
      self.decoder = build_decoder(latent_size, size_before_flat, input_size)

  def call(self, data):
    latent_vector = self.encoder(data)
    reconstructed = self.decoder(latent_vector)
    return reconstructed

  def train_step(self, data):
      with tf.GradientTape() as tape:
        reconstructed_img = self.call(data)
        mse = keras.losses.mse(data, reconstructed_img)
        total_loss = mse
      grads = tape.gradient(total_loss, self.trainable_weights)
      self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
      return {
          "mse": mse
      }

In [ ]:
# file path for pretrained autoencoder
ae_fp = "./gdrive/My Drive/Faces/Trained_ae"

In [ ]:
# training constants
LEARNING_RATE = 0.0005
NUM_EPOCHS = 7

optimizer = keras.optimizers.Adam(lr = LEARNING_RATE)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
# join the two models
autoencoder_network = AE(IMG_SIZE, LATENT_VECTOR_SIZE)
if os.path.isdir(ae_fp):
  train_autoencoder = False
  # load pre-trained model
  print("loaded old")
  encoder = keras.models.load_model(os.path.join(ae_fp, "encoder"))
  decoder = keras.models.load_model(os.path.join(ae_fp, "decoder"))
  autoencoder_network.encoder = encoder
  autoencoder_network.decoder = decoder
else:
  train_autoencoder = True
  print("created new")
  autoencoder_network.compile(optimizer=optimizer, loss=keras.losses.mean_squared_error)

loaded old


In [ ]:
#train_autoencoder = True
if train_autoencoder:
  autoencoder_network.fit(train_dataset, 
                        shuffle=True, 
                        epochs = NUM_EPOCHS,
                        steps_per_epoch= train_dataset.samples // TRAIN_BATCH_SIZE,
                        validation_data=val_dataset,
                        validation_steps=val_dataset.samples // VAL_BATCH_SIZE)
  # save network
  autoencoder_network.encoder.save(os.path.join(ae_fp, "encoder"))
  autoencoder_network.decoder.save(os.path.join(ae_fp, "decoder"))

In [ ]:
import matplotlib.pyplot as plt
IMGS_PER_ROW = 4

# function to display a batch of images
def display_imgs(batch, from_img=True):
  # define plt stuff
  if from_img:
    grid_size = VAL_BATCH_SIZE * 2 // IMGS_PER_ROW
  else:
    grid_size = VAL_BATCH_SIZE // IMGS_PER_ROW
  fig, axes = plt.subplots(grid_size, IMGS_PER_ROW, figsize=IMG_SIZE[:2])
  axes = axes.flatten()

  for img, ax in zip(batch, axes):
    ax.imshow(img)
    ax.axis('off')
  plt.tight_layout()
  plt.show()
# visualise network's reconstruction of image
def reconstruct_image_from_image(input_imgs, encoder, decoder):
  #print(input_imgs.shape)
  # get output of network
  predictions = decoder.predict(encoder.predict(input_imgs))
  #print(predictions.shape)

  # interleave actual and predicted
  interleaved = []
  for i in range(VAL_BATCH_SIZE):
    interleaved.append(input_imgs[i])
    interleaved.append(predictions[i])
  interleaved = np.array(interleaved)

  # display images
  display_imgs(interleaved)

In [ ]:
reconstruct_image_from_image(val_dataset[0], autoencoder_network.encoder, autoencoder_network.decoder)

In [ ]:
# dataset for latents
latents_dataset, _ = build_dataset(IMGS_FP, 1, VAL_BATCH_SIZE, shuffle=False, validation_split=0)

Found 202592 images belonging to 1 classes.
Found 0 images belonging to 1 classes.


In [ ]:
# get list of latent vectors
latents = [autoencoder_network.encoder(latents_dataset[i])[0] for i in range(128000)]

In [ ]:
print(np.array(latents).shape)

(128000, 200)


In [ ]:
# do PCA
from sklearn.decomposition import PCA
mean_vector = np.mean(latents, axis=0)
norm_dense_rep = latents - mean_vector
pca = PCA(n_components=norm_dense_rep.shape[1])
pca.fit(norm_dense_rep)
values = np.sqrt(pca.explained_variance_)
vectors = pca.components_

In [ ]:
saving_latents = True
if saving_latents:
  latents_fp = "./gdrive/My Drive/Faces/latents"
  np.save(os.path.join(latents_fp, "eigenvalues.npy"), values)
  np.save(os.path.join(latents_fp, "eigenvectors.npy"), vectors)
  np.save(os.path.join(latents_fp, "latents_128k.npy"), latents)